# Поиск оптимальной марковской цепи из 2 состояний (вероятности перехода p и q) для последовательности из 0 и 1

## Необходимые функции

p и q - вероятности, числа в интервале (0, 1). $C(\mu) = $ # знаков после запятой

In [ ]:
# функция для перевода десятичных дробей в двоичные
def binFloat(n:float,k:int=30) -> str: # n - число, k - сколько знаков будет в двоичном числе суммарно
    s = bin(int(n))[2:] + '.'# перевели целую часть в двоичную систему
    r = n - int(n) # дробная часть n
    while len(s) < k+1 and r > 0:
        #print(r)
        r *= 2
        if r >= 1:
            s += '1'
            r -= 1
        else:
            s += '0'
    s += '0' * (k-len(s)+1) # дописали недостающие нули до заданной длины k в конец
    return s

In [ ]:
# функция для перевода десятичных дробей в двоичные
def binFloatPure(n:float,k:int=30) -> str: # отличие - не дописывает недостающие нули в конец, n - число, k - сколько знаков будет в двоичном числе суммарно
    s = bin(int(n))[2:] + '.'# перевели целую часть в двоичную систему
    r = n - int(n) # дробная часть n
    while len(s) < k+1 and r > 0:
        #print(r)
        r *= 2
        if r >= 1:
            s += '1'
            r -= 1
        else:
            s += '0'
    #s += '0' * (k-len(s)+1) # дописали недостающие нули до заданной длины k в конец
    return s

In [ ]:
# пример работы
binFloat(0.125,10)

'0.001000000'

In [ ]:
# наоборот - дробное двоичное число в десятичную дробь
def floatBin(s:str) -> float:
    if '.' not in s:
        return int(s,2)

    a,b = s.split('.')
    res = int(a,2)
    for i in range(1,len(b)+1):
        res += int(b[i-1]) * 2**(-i)
    return res

In [ ]:
# пример работы
from math import pi
from math import sqrt
print(round(pi, 10))
print(binFloat(pi,30))

3.1415926536
11.0010010000111111011010101000


In [ ]:
print(binFloat(sqrt(2)))
print(binFloat(sqrt(3)))

1.01101010000010011110011001100
1.10111011011001111010111010000


In [ ]:
print(floatBin('11.0010010000111111011010101000'))
print('совпало с приближением пи')

3.141592651605606
совпало с приближением пи


In [ ]:
def digrams(s:str) -> tuple: # принимает на вход строку и возвращает кол-во диграмм каждого типа
    s = s.replace('.','')
    n00 = n01 = n10 = n11 = 0
    for i in range(len(s)-1):
        if s[i] == s[i+1] == '1':
            n11 += 1
        elif s[i] == s[i+1] == '0':
            n00 += 1
        elif s[i] == '0' and s[i+1] == '1':
            n01 +=1
        elif s[i] == '1' and s[i+1] == '0':
            n10 += 1
    return (n00, n01, n10, n11)

In [ ]:
from math import log2
from math import sqrt
from math import pi
from itertools import product
import time

def logProb(s:str,p:float,q:float) -> float: # возвращает log2 вероятности получить строку s с переходными вероятностями p из 0 в 1 И q из 1 в 0
    s = s.replace('.','')
    n00, n01, n10, n11 = digrams(s)
    # тут нужны допущения на log2 при 0 и 1, чтобы python не ругался на математику при подсчёте log2(0)
    if p == 0:
        log_p = -1e10 
        log_1_p = 0
    elif p == 1:
        log_1_p = -1e10 
        log_p = 0
    else:
        log_p = log2(p)
        log_1_p = log2(1-p)
    if q == 0:
        log_q = -1e10 
        log_1_q = 0
    elif q == 1:
        log_1_q = -1e10 
        log_q = 0
    else:
        log_q = log2(q)
        log_1_q = log2(1-q)
    log_q = -1e10 if q == 0 else log2(q)

    # собственно, вероятность получить строку s
    res = n01*log_p + n10*log_q + n00*log_1_p + n11*log_1_q
    return res

# v 1.1

def MDL(s:str,k:int,l:int) -> tuple: 
    m = 10**30 # минимум пусть сначала будет таким большим, чтобы было, с чем сравнивать
    # генерируем все возможные двоичные числа в [0, 1] с k и l знаков после запятой
    p_range = ['0.'+''.join(i) for i in product('01',repeat=k)] + ['1.']
    q_range = ['0.'+''.join(i) for i in product('01',repeat=l)] + ['1.']
    for p in p_range:
        for q in q_range:
            lp = len(p)-1
            lq = len(p)-1
            pf = floatBin(p)
            qf = floatBin(q)
            #r = lp+lq-logProb(s,pf,qf) # Rissanen complexity
            r = -logProb(s,pf,qf)
            if r < m: # условие обновления минимума и оптимальных p и q
                m = r
                p0 = pf
                q0 = qf
    return (p0, q0, m) # выводит minimun description length, p, q
    

In [ ]:
def MDLbin(s,k,l): #возвращает двоичные числа
    m = 10**30 # минимум пусть сначала будет таким большим, чтобы было, с чем сравнивать
    lgm = 10**30
    # генерируем все возможные двоичные числа в [0, 1] с k и l знаков после запятой
    p_range = ['0.'+''.join(i) for i in product('01',repeat=k)] + ['1.']
    q_range = ['0.'+''.join(i) for i in product('01',repeat=l)] + ['1.']
    for p in p_range:
        for q in q_range:
            pf = floatBin(p)
            qf = floatBin(q)
            lg = -logProb(s,pf,qf)
            r = k+l+lg # Rissanen complexity
            if r < m: # условие обновления минимума и оптимальных p и q
                m = r
                lgm = lg
                p0 = p
                q0 = q
    return (m, lgm, p0, q0) # выводит log(1/m(x)), p, q

In [ ]:
def tablePrint(arr, rnd = 4, range_from = 1, range_to = 6, isBin = True):
    print('\\begin{table}[h]')
    smpl = list(range(range_from,range_to+1))
    if isBin:
        print('\\caption{Таблица оптимальных зн-й p и q в двоичной записи для $\pi$}')
    else:
        print('\\caption{Таблица оптимальных значений p и q для $\pi$}')
        
    print('\\label{sometable}')
    print('\\begin{center}')
    print('\\begin{tabular}{|' + 'l|' * (len(smpl) + 1) + '}')
    print('\\hline')
    print('k / l &' + ' & '.join([str(i) for i in smpl]) + '\\\\')
    ind = 0
    print('\\hline')
    for i in arr:
        print(smpl[ind],end = ' ')
        for k in range(4):
            for j in i:
                if isBin:
                    print('&', round(j[k],rnd), end='') if k == 0 or k == 1 else print('&', j[k], end='')
                else:
                    print('&', round(j[k],rnd), end='')
            print('\\\\')
        print('\\hline')
        ind += 1
    print('\\end{tabular}')
    print('\\end{center}')
    print('\\end{table}')

In [ ]:
def tableMDL(val,range_from = 1, range_to = 6,setRnd = 4,isBin = True):
    s = binFloat(val,30)
    smpl = list(range(range_from,range_to+1))
    n = len(smpl)
    arr = []

    if isBin:
        for k in smpl:
            arr.append([])
            for l in smpl:
                arr[k-smpl[0]].append(MDLbin(s,k,l))

        d = digrams(s)
        print(f"Колическо диграмм: $n(00) = {str(d[0])}, n(01) = {str(d[1])}, n(10) = {str(d[2])}, n(11) = {str(d[3])}$")
        tablePrint(arr, rnd = setRnd)
    else:
        print('to do')


##1) '001'*300

1) 001001...0010 (длина 3001)

In [ ]:
s = '001' * 1000 + '0'
print('Mimimum description lenght, p, q:')
MDL(s,5,5)

Mimimum description lenght, p, q:


(0.5, 1, 2000.0)

In [ ]:
MDLbin(s,5,5)

(2010.0, 2000.0, '0.10000', '1.')

In [ ]:
digrams(s)

(1000, 1000, 1000, 0)

Как и ожидалось по аналитическим выкладкам

##2) $\pi\approx3,14159265$

In [ ]:
tableMDL(pi)

Колическо диграмм: $n(00) = 7, n(01) = 7, n(10) = 8, n(11) = 7$
\begin{table}[h]
\caption{Таблица оптимальных зн-й p и q в двоичной записи для $\pi$}
\label{sometable}
\begin{center}
\begin{tabular}{|l|l|l|l|l|l|l|}
\hline
k / l &1 & 2 & 3 & 4 & 5 & 6\\
\hline
1 & 31.0& 32.0& 33.0& 33.9891& 34.9521& 35.9521\\
& 29.0& 29.0& 29.0& 28.9891& 28.9521& 28.9521\\
& 0.1& 0.1& 0.1& 0.1& 0.1& 0.1\\
& 0.1& 0.10& 0.100& 0.1001& 0.10001& 0.100010\\
\hline
2 & 32.0& 33.0& 34.0& 34.9891& 35.9521& 36.9521\\
& 29.0& 29.0& 29.0& 28.9891& 28.9521& 28.9521\\
& 0.10& 0.10& 0.10& 0.10& 0.10& 0.10\\
& 0.1& 0.10& 0.100& 0.1001& 0.10001& 0.100010\\
\hline
3 & 33.0& 34.0& 35.0& 35.9891& 36.9521& 37.9521\\
& 29.0& 29.0& 29.0& 28.9891& 28.9521& 28.9521\\
& 0.100& 0.100& 0.100& 0.100& 0.100& 0.100\\
& 0.1& 0.10& 0.100& 0.1001& 0.10001& 0.100010\\
\hline
4 & 34.0& 35.0& 36.0& 36.9891& 37.9521& 38.9521\\
& 29.0& 29.0& 29.0& 28.9891& 28.9521& 28.9521\\
& 0.1000& 0.1000& 0.1000& 0.1000& 0.1000& 0.1000\\
& 0.1& 0.10& 0

In [ ]:
s = binFloat(pi,30)
d = digrams(s)
s_print = f"Колическо диграмм: $n(00) = {str(d[0])}, n(01) = {str(d[1])}, n(10) = {str(d[2])}, n(11) = {str(d[3])}$"
print(s_print)

Колическо диграмм: $n(00) = 7, n(01) = 7, n(10) = 8, n(11) = 7$


In [ ]:
# оптимальное значение для Pi с диграммами 7,7,8,7
p0 = d[1] / (d[1] + d[0]) 
q0 = d[2] / (d[2] + d[3]) 
print('p:',p0,binFloat(p0))
print('q:',q0,binFloat(q0))

p: 0.5 0.10000000000000000000000000000
q: 0.5333333333333333 0.10001000100010001000100010001


#3) $\sqrt{2}$

In [ ]:
s = binFloat(sqrt(2),30)
print(s)
d = digrams(s)
print(d)

1.01101010000010011110011001100
(8, 7, 8, 6)


In [ ]:
p0 = d[1] / (d[1] + d[0]) 
q0 = d[2] / (d[2] + d[3]) 
print('p:',p0,binFloat(p0))
print('q:',q0,binFloat(q0))

p: 0.4666666666666667 0.01110111011101110111011101110
q: 0.5714285714285714 0.10010010010010010010010010010


In [ ]:
for i in range(1,8):
    print(i,MDL(s,i,i)[:2],MDLbin(s,i,i)[2:])

1 (0.5, 0.5) ('0.1', '0.1')
2 (0.5, 0.5) ('0.10', '0.10')
3 (0.5, 0.625) ('0.100', '0.101')
4 (0.4375, 0.5625) ('0.0111', '0.1001')
5 (0.46875, 0.5625) ('0.01111', '0.10010')
6 (0.46875, 0.578125) ('0.011110', '0.100101')
7 (0.46875, 0.5703125) ('0.0111100', '0.1001001')


In [ ]:
print(MDL(s,15,1)[:2],MDLbin(s,15,1)[2:])

(0.4666748046875, 0.5) ('0.011101110111100', '0.1')


In [ ]:
# d = (n00, n01, n10, n11)
trgt = d[1]*log2(p0) + d[0]*log2(1-p0)
f = lambda x: d[1]*log2(x) + d[0]*log2(1-x)
f1 = f(floatBin('0.011110'))
print(f'true min log: {trgt}')
print(f1, abs(trgt-f1))
f2 = f(floatBin('0.011101'))
print(f2, abs(trgt-f2))

true min log: -14.951874479724548
-14.952063100737655 0.00018862101310723745
-14.959868898547263 0.007994418822715232


In [ ]:
print(floatBin('0.011110'),floatBin('0.011101'))

0.46875 0.453125


In [ ]:
print(binFloat(59/128,8),binFloat(60/128,8))

0.0111011 0.0111100


In [ ]:
d[1] / (d[1] + d[0]) * 2**7

59.733333333333334

Получили результат, идетичный числу пи

In [ ]:
tableMDL(sqrt(2))

Колическо диграмм: $n(00) = 8, n(01) = 7, n(10) = 8, n(11) = 6$
\begin{table}[h]
\caption{Таблица оптимальных зн-й p и q в двоичной записи для $\pi$}
\label{sometable}
\begin{center}
\begin{tabular}{|l|l|l|l|l|l|l|}
\hline
k / l &1 & 2 & 3 & 4 & 5 & 6\\
\hline
1 & 31.0& 32.0& 32.9148& 33.7965& 34.7965& 35.795\\
& 29.0& 29.0& 28.9148& 28.7965& 28.7965& 28.795\\
& 0.1& 0.1& 0.1& 0.1& 0.1& 0.1\\
& 0.1& 0.10& 0.101& 0.1001& 0.10010& 0.100101\\
\hline
2 & 32.0& 33.0& 33.9148& 34.7965& 35.7965& 36.795\\
& 29.0& 29.0& 28.9148& 28.7965& 28.7965& 28.795\\
& 0.10& 0.10& 0.10& 0.10& 0.10& 0.10\\
& 0.1& 0.10& 0.101& 0.1001& 0.10010& 0.100101\\
\hline
3 & 33.0& 34.0& 34.9148& 35.7965& 36.7965& 37.795\\
& 29.0& 29.0& 28.9148& 28.7965& 28.7965& 28.795\\
& 0.100& 0.100& 0.100& 0.100& 0.100& 0.100\\
& 0.1& 0.10& 0.101& 0.1001& 0.10010& 0.100101\\
\hline
4 & 33.9891& 34.9891& 35.9039& 36.7856& 37.7856& 38.7842\\
& 28.9891& 28.9891& 28.9039& 28.7856& 28.7856& 28.7842\\
& 0.0111& 0.0111& 0.0111& 0.0111& 0

#4) $\sqrt{3}$

In [ ]:
d = digrams(s)
print(d)

(8, 7, 8, 6)


In [ ]:
p0 = d[1] / (d[1] + d[0]) 
q0 = d[2] / (d[2] + d[3]) 
print('p:',p0,binFloat(p0))
print('q:',q0,binFloat(q0))

p: 0.4666666666666667 0.01110111011101110111011101110
q: 0.5714285714285714 0.10010010010010010010010010010


Здесь результат уже немного отличается, причём Minimal description length похожая: 46, 43.

In [ ]:
tableMDL(sqrt(3))

Колическо диграмм: $n(00) = 4, n(01) = 7, n(10) = 8, n(11) = 10$
\begin{table}[h]
\caption{Таблица оптимальных зн-й p и q в двоичной записи для $\pi$}
\label{sometable}
\begin{center}
\begin{tabular}{|l|l|l|l|l|l|l|}
\hline
k / l &1 & 2 & 3 & 4 & 5 & 6\\
\hline
1 & 31.0& 32.0& 33.0& 33.8419& 34.8419& 35.8419\\
& 29.0& 29.0& 29.0& 28.8419& 28.8419& 28.8419\\
& 0.1& 0.1& 0.1& 0.1& 0.1& 0.1\\
& 0.1& 0.10& 0.100& 0.0111& 0.01110& 0.011100\\
\hline
2 & 31.9053& 32.9053& 33.9053& 34.7472& 35.7472& 36.7472\\
& 28.9053& 28.9053& 28.9053& 28.7472& 28.7472& 28.7472\\
& 0.11& 0.11& 0.11& 0.11& 0.11& 0.11\\
& 0.1& 0.10& 0.100& 0.0111& 0.01110& 0.011100\\
\hline
3 & 32.4067& 33.4067& 34.4067& 35.2486& 36.2486& 37.2486\\
& 28.4067& 28.4067& 28.4067& 28.2486& 28.2486& 28.2486\\
& 0.101& 0.101& 0.101& 0.101& 0.101& 0.101\\
& 0.1& 0.10& 0.100& 0.0111& 0.01110& 0.011100\\
\hline
4 & 33.4067& 34.4067& 35.4067& 36.2486& 37.2486& 38.2486\\
& 28.4067& 28.4067& 28.4067& 28.2486& 28.2486& 28.2486\\
& 0.1010& 

# Оптимальные значения для заданного количества диграмм

In [ ]:
a = list(map(binFloat,[pi,sqrt(2),sqrt(3)]))
print(a)
print(binFloat(pi,50))

['11.0010010000111111011010101000', '1.01101010000010011110011001100', '1.10111011011001111010111010000']
11.001001000011111101101010100010001000010110100011


In [ ]:
for s in a:
    print(*digrams(s), sep = ' & ')

30 & 7 & 7 & 8 & 7
30 & 8 & 7 & 8 & 6
30 & 4 & 7 & 8 & 10


Берём при фиксированном n(00) = 1 разные значения n(01) и ищем оптимальное k в каждом случае. $f_1(p) = n(01)\log_2{p} + n(00)\log_2{(1-p)}$, точка максимума $p=\frac{n(01)}{n(01)+n(00)}$.

In [ ]:
n00 = 1
arr = []
for i in range(2,11):
    m = 10**30 # это точно не будет минимумом
    l = 10**30
    n01 = 2**i # 2,4,8,16
    p0 = n01 / (n00 + n01)
    a=[]
    for k in range(1,1001):
        pb = binFloat(p0,k+1) # нижнее округление
        p = floatBin(pb)
        #print(p0,pb,p)
        lg = - (n01*log2(p) + n00*log2(1-p)) 
        dl = k +lg
        a.append((round(lg,4),round(dl,4)))
        #print(n01,k,round(lg,4),round(dl,4),end = '\n')
        if dl < m:
            m = dl
            k0 = k
            pb0= pb
            lg0 = lg
        if lg < l-1:
            lg1 = lg
            k1 = k
            pb1 = pb

        pb = binFloat(p0 + 2**(-k),k+1) # нижнее округление
        p = floatBin(pb)
        #print(p0,pb,k,2**(-k),p)
        lg = - (n01*log2(p) +n00*log2(1-p)) if 0 < p < 1 else 10**30
        dl = k +lg
        a.append((round(lg,4),round(dl,4)))
        #print(n01,k,round(lg,4),round(dl,4),end = '\n')
        if dl < m:
            m = dl
            k0 = k
            pb0= pb
            lg0 = lg
        if lg < l-1:
            lg1 = lg
            k1 = k
            pb1 = pb

    arr.append((n01,k,m))
    print(n00,n01,k0,pb0,round(p,4),round(lg0,4),round(m,4),sep = ' & ', end = ' \\\\ \n \\hline \n')

1 & 4 & 2 & 0.11 & 0.8 & 3.6601 & 5.6601 \\ 
 \hline 
1 & 8 & 2 & 0.11 & 0.8889 & 5.3203 & 7.3203 \\ 
 \hline 
1 & 16 & 3 & 0.111 & 0.9412 & 6.0823 & 9.0823 \\ 
 \hline 
1 & 32 & 4 & 0.1111 & 0.9697 & 6.9795 & 10.9795 \\ 
 \hline 
1 & 64 & 5 & 0.11111 & 0.9846 & 7.9314 & 12.9314 \\ 
 \hline 
1 & 128 & 6 & 0.111111 & 0.9922 & 8.9082 & 14.9082 \\ 
 \hline 
1 & 256 & 7 & 0.1111111 & 0.9961 & 9.8967 & 16.8967 \\ 
 \hline 
1 & 512 & 8 & 0.11111111 & 0.9981 & 10.891 & 18.891 \\ 
 \hline 
1 & 1024 & 9 & 0.111111111 & 0.999 & 11.8882 & 20.8882 \\ 
 \hline 


In [ ]:
n00 = 100000
arr = []
for i in range(2,11):
    m = 10**30 # это точно не будет минимумом
    lg1 = 10**30
    n01 = n00*i # 2,4,8,16
    p0 = n01 / (n00 + n01)
    a=[]
    for k in range(1,1001):
        pb = binFloat(p0,k+1) # нижнее округление
        p = floatBin(pb)
        #print(p0,pb,p)
        lg = - (n01*log2(p) + n00*log2(1-p)) 
        dl = k +lg
        a.append((round(lg,4),round(dl,4)))
        #print(n01,k,round(lg,4),round(dl,4),end = '\n')
        if dl < m:
            m = dl
            k0 = k
            pb0= pb
            lg0 = lg
        if lg < lg1:
            lg1 = lg
            k1 = k
            pb1 = pb

        pb = binFloat(p0 + 2**(-k),k+1) # нижнее округление
        p = floatBin(pb)
        #print(p0,pb,k,2**(-k),p)
        lg = - (n01*log2(p) +n00*log2(1-p)) if 0 < p < 1 else 10**30
        dl = k +lg
        a.append((round(lg,4),round(dl,4)))
        #print(n01,k,round(lg,4),round(dl,4),end = '\n')
        if dl < m:
            m = dl
            k0 = k
            pb0= pb
            lg0 = lg
        if lg < lg1:
            lg1 = lg
            k1 = k
            pb1 = pb
            dl1 = dl

    arr.append((n01,k,m))
    print(n00,n01,k0,pb0,round(p,4),round(lg0,4),round(m,4),k1,round(lg1,4),sep = ' & ', end = ' \\\\ \n \\hline \n')

100000 & 200000 & 9 & 0.101010101 & 0.6667 & 275489.1627 & 275498.1627 & 29 & 275488.7502 \\ 
 \hline 
100000 & 300000 & 2 & 0.11 & 0.75 & 324511.2498 & 324513.2498 & 2 & 324511.2498 \\ 
 \hline 
100000 & 400000 & 8 & 0.11001101 & 0.8 & 360965.426 & 360973.426 & 26 & 360964.0474 \\ 
 \hline 
100000 & 500000 & 9 & 0.110101011 & 0.8333 & 390014.7766 & 390023.7766 & 34 & 390013.453 \\ 
 \hline 
100000 & 600000 & 9 & 0.110110111 & 0.8571 & 414171.2664 & 414180.2664 & 31 & 414170.945 \\ 
 \hline 
100000 & 700000 & 3 & 0.111 & 0.875 & 434851.5546 & 434854.5546 & 30 & 434851.5546 \\ 
 \hline 
100000 & 800000 & 9 & 0.111000111 & 0.8889 & 452932.8105 & 452941.8105 & 27 & 452932.5013 \\ 
 \hline 
100000 & 900000 & 9 & 0.111001101 & 0.9 & 468996.8194 & 469005.8194 & 30 & 468995.5936 \\ 
 \hline 
100000 & 1000000 & 10 & 0.1110100011 & 0.9091 & 483446.7613 & 483456.7613 & 33 & 483446.6856 \\ 
 \hline 


In [ ]:
l1 = [275489.1627,360965.426,390014.7766,414171.2664,434854.5546,452932.8105,468996.8194,483446.7613]
l2 = [275488.7502,360964.0474,390013.453,414170.945,434851.5546,452932.5013,468995.5936,483446.6856]
arr = [((l1[i]/l2[i]) - 1)*100 for i in range(len(l1))]
print(max(arr),arr)

0.0006898906002028582 [0.00014973388193784132, 0.0003819216927380964, 0.00033937290875307724, 7.760080804608549e-05, 0.0006898906002028582, 6.826624256994762e-05, 0.0002613670611495422, 1.565839673478564e-05]


In [ ]:
def tablePrint2(arr, rnd = 4):
    print('\\begin{table}[h]')
    smpl = list(range(range_from,range_to+1))
    if isBin:
        print('\\caption{Таблица оптимальных зн-й p и q в двоичной записи для $\pi$}')
    else:
        print('\\caption{Таблица оптимальных значений p и q для $\pi$}')
        
    print('\\label{sometable}')
    print('\\begin{center}')
    print('\\begin{tabular}{|' + 'l|' * (len(smpl) + 1) + '}')
    print('\\hline')
    print('k / l &' + ' & '.join([str(i) for i in smpl]) + '\\\\')
    ind = 0
    print('\\hline')
    for i in arr:
        print(smpl[ind],end = ' ')
        for k in range(4):
            for j in i:
                if isBin:
                    print('&', round(j[k],rnd), end='') if k == 0 or k == 1 else print('&', j[k], end='')
                else:
                    print('&', round(j[k],rnd), end='')
            print('\\\\')
        print('\\hline')
        ind += 1
    print('\\end{tabular}')
    print('\\end{center}')
    print('\\end{table}')

Здесь уже учитываем свойство независимости.

In [ ]:
def optimalMDL(s,beg=1,en=10):
    m = 10**30 # минимум пусть сначала будет таким большим, чтобы было, с чем сравнивать
    for k in range(beg,en):
        for l in range(beg,en):
            mkl = (MDLbin(s,k,l),k,l)
            if mkl[0][0] < m:
                m = mkl[0][0]
                opt = mkl
    return opt

In [ ]:
s = binFloat(pi)
optimalMDL(s,1,11)

((31.951875216775942, 27.951875216775942, '0.1', '0.100010001'), 1, 9)

In [ ]:
def optimalMDL2(s,beg=0,en=10):
    m = 10**30 # минимум пусть сначала будет таким большим, чтобы было, с чем сравнивать
    for k in range(beg,en):
        l = 1
        # генерируем все возможные двоичные числа в [0, 1] с k и l знаков после запятой
        p_range = ['0.'+''.join(i) for i in product('01',repeat=k)] + ['1.']
        q_range = ['0.'+''.join(i) for i in product('01',repeat=l)] + ['1.']
        for p in p_range:
            for q in q_range:
                lp = len(p)-1
                lq = len(p)-1
                pf = floatBin(p)
                qf = floatBin(q)
                #r = lp+lq-logProb(s,pf,qf) # Rissanen complexity
                r = -logProb(s,pf,qf)
                if r < m: # условие обновления минимума и оптимальных p и q
                    m = r
                    p0 = pf
                    q0 = qf
        # так мы нашли оптимальное p длины k, теперь оптимизируем l
        for l in range(beg,en):

# Марковские цепи с 4 состояниями

случайное блуждание

In [ ]:
import random
from random import randint
from random import choices
def randomWalk(P:list,d:dict,k:int)->str: #P- матрица переходных вероятностей, d - словарь, какому номеру какой код соответсвует, k - длина блуждания
    state = randint(0,3) # начальное состояние
    s = d[state]
    for i in range(k):
        state = choices([0,1,2,3],weights = P[state],k=1)[0] # в какую позицию пойдём дальше, всего 4 варианта: 0,1,2,3
        s += d[state]
    return s

In [ ]:
random.seed(892)
P = [[1/4,1/4,1/2,0],[1/2,1/4,0,1/4],[1/4,1/2,0,1/4],[1/4,0,1/4,1/2]]
d = {0:'0',1:'0',2:'1',3:'1'}
k = 20
s = randomWalk(P,d,k)
#print(s[:10],'...',sep='')
print(P)
print(s)

[[0.25, 0.25, 0.5, 0], [0.5, 0.25, 0, 0.25], [0.25, 0.5, 0, 0.25], [0.25, 0, 0.25, 0.5]]
011110100010000111110


In [ ]:
k = 10
s = randomWalk(P,d,k)
#print(s[:10],'...',sep='')
print(s) # s = 01001111010

01001111010


получили некоторую реализацию, теперь подберём оптимальную цепь из 4 состояний

In [ ]:
# вероятность получить такую цепочку
def prob(P:list,d:dict,s:str,n:int=0,state:int = None)->float:# P- матрица переходных вероятностей, d - словарь, какому символу какие состояния соотвествуют,
# s - цепочка из 0 и 1, n - позиция, state - в каком состоянии сейчас марковская цепь
    #print(s,n,s[n],d[s[n]])
    if n == len(s) - 1:
        return sum(P[state][i] for i in d[s[n]])
    if n == 0:
        k = 1/len(d[s[n]])
        return k * sum(prob(P,d,s,n+1,i) for i in d[s[n]])
    else:
        #print(state)#, [P[state][i] for i in d[s[n]]])
        return sum(P[state][i]*prob(P,d,s,n+1,i) for i in d[s[n]])

In [ ]:
prob(P,d,'011') # пример верный

0.15625

In [ ]:
s = '01001111010'
print(s)
P = [[1/4,1/4,1/2,0],[1/2,1/4,0,1/4],[1/4,1/2,0,1/4],[1/4,0,1/4,1/2]]
d = {'0':[0,1],'1':[2,3]}
start = time.time()
print(prob(P,d,s))
end = time.time() - start
print('time:',end)

01001111010
0.0008563995361328125
time: 0.0060214996337890625


In [ ]:
# поблуждаем 1000 раз и выберем самую часто встречающуюся строку
import random
random.seed(892)
P = [[1/4,1/4,1/2,0],[1/2,1/4,0,1/4],[1/4,1/2,0,1/4],[1/4,0,1/4,1/2]]
dic = {}
d = {0:'0',1:'0',2:'1',3:'1'}
k = 10
for i in range(1000000):
    s1 = randomWalk(P,d,k)
    if s1 in dic:
        dic[s1] += 1
    else:
        dic[s1] = 1
ans = dict(sorted(dic.items(), key=lambda item: item[1], reverse = True))
print(ans)

{'00000000000': 3273, '11111111111': 2780, '10000000000': 2721, '00100000000': 2458, '00001000000': 2419, '00000000100': 2405, '00000010000': 2402, '00000001000': 2368, '00000100000': 2329, '00010000000': 2299, '00000000010': 2243, '00000000001': 2189, '10000010000': 2049, '10000000100': 2012, '10000100000': 2001, '01000000000': 1982, '10001000000': 1975, '10010000000': 1927, '10000001000': 1875, '10000000010': 1857, '11111111110': 1846, '00100100000': 1845, '11111100000': 1815, '11100000000': 1811, '11111110000': 1797, '10000000001': 1790, '11111111000': 1784, '00010000100': 1775, '00100001000': 1749, '00010010000': 1744, '11111000000': 1728, '11110000000': 1720, '00001000100': 1717, '11111111100': 1716, '00100010000': 1709, '00001001000': 1704, '00000100100': 1697, '00000010010': 1695, '00010001000': 1693, '00100000100': 1691, '10100000000': 1616, '00100000010': 1608, '00000100001': 1608, '00100000001': 1603, '00010000010': 1599, '00001000010': 1586, '00000001001': 1581, '00000100010

In [ ]:
from math import log
P = [[1/4,1/4,1/2,0],[1/2,1/4,0,1/4],[1/4,1/2,0,1/4],[1/4,0,1/4,1/2]]
d = {'0':[0,1],'1':[2,3]}
dl = sum(len(binFloatPure(i)) for i in P[0]+P[1]+P[2]+P[3]) - 2*len(P)*len(P)
print('Блуждание\tЧастота\tВероятность по матрице\tDescription Length')
for i in list(ans.keys())[:15]:
    pr = prob(P,d,i)
    mdl = dl - log(pr,2) #if pr > 0 else -10**6
    print(i,ans[i],pr,mdl,sep='\t')

Блуждание	Частота	Вероятность по матрице	Description Length
00000000000	3273	0.0066089630126953125	27.241360362975424
11111111111	2780	0.005475044250488281	27.51291365972873
10000000000	2721	0.005475044250488281	27.51291365972873
00100000000	2458	0.0048580169677734375	27.685416755725804
00001000000	2419	0.0047454833984375	27.7192292298694
00000000100	2405	0.004791259765625	27.705379251108372
00000010000	2402	0.0047435760498046875	27.719809207783094
00000001000	2368	0.004733562469482422	27.722857922966305
00000100000	2329	0.004741191864013672	27.720534508206036
00010000000	2299	0.004722118377685547	27.72635007605631
00000000010	2243	0.004455089569091797	27.810329847396247
00000000001	2189	0.00434112548828125	27.84771515769342
10000010000	2049	0.00392913818359375	27.991571377929418
10000000100	2012	0.0039691925048828125	27.976938750264665
10000100000	2001	0.0039310455322265625	27.990871210460742


Получили тривиальные последовательности, оставляем исходную, сгенерированную случайным блужанием.

In [ ]:
P = [[1/4,1/4,1/2,0],[1/2,1/4,0,1/4],[1/4,1/2,0,1/4],[1/4,0,1/4,1/2]]
d = {'0':[0,1],'1':[2,3]}
dl = sum(len(binFloatPure(i)) for i in P[0]+P[1]+P[2]+P[3]) - 2*len(P)*len(P)
s = '01001111010'
pr = prob(P,d,s)
mdl = dl - log(pr,2) #if pr > 0 else -10**6
print(s,pr,mdl)

01001111010 0.0008563995361328125 30.189428365258856


In [ ]:
print(list(binFloatPure(i) for i in P[0]+P[1]+P[2]+P[3]))
dl = sum(len(binFloatPure(i)) for i in P[0]+P[1]+P[2]+P[3]) - 2*len(P)*len(P)
print(dl)

['0.01', '0.01', '0.1', '0.', '0.1', '0.01', '0.', '0.01', '0.01', '0.1', '0.', '0.01', '0.01', '0.', '0.01', '0.1']
20


Теперь переберём различные модели из 4 состояний. Сначала при том же словаре, но при разных переходных вероятностях. Пусть вероятности - числа от 0 до 1 включительно с шагом 0.1, в каждой из 4 строк мы задаём 3 переходных вероятности, четвёртую вычисляем из соображения о том, что в сумме по строке 4 вероятности дают 1. Таким образом, в матрице переходных вероятностей P у нас 12 свободных переменных

In [ ]:
from itertools import product
k=2
p_range = ['0.'+''.join(i) for i in product('01',repeat=k)] + ['1.']
np = len(p_range)
prob_sep = []
for i1 in range(np):
    for i2 in range(i1,np):
        for i3 in range(i2,np):
            prob_sep.append([p_range[i1],p_range[i2],p_range[i3]])
print(prob_sep)

[['0.00', '0.00', '0.00'], ['0.00', '0.00', '0.01'], ['0.00', '0.00', '0.10'], ['0.00', '0.00', '0.11'], ['0.00', '0.00', '1.'], ['0.00', '0.01', '0.01'], ['0.00', '0.01', '0.10'], ['0.00', '0.01', '0.11'], ['0.00', '0.01', '1.'], ['0.00', '0.10', '0.10'], ['0.00', '0.10', '0.11'], ['0.00', '0.10', '1.'], ['0.00', '0.11', '0.11'], ['0.00', '0.11', '1.'], ['0.00', '1.', '1.'], ['0.01', '0.01', '0.01'], ['0.01', '0.01', '0.10'], ['0.01', '0.01', '0.11'], ['0.01', '0.01', '1.'], ['0.01', '0.10', '0.10'], ['0.01', '0.10', '0.11'], ['0.01', '0.10', '1.'], ['0.01', '0.11', '0.11'], ['0.01', '0.11', '1.'], ['0.01', '1.', '1.'], ['0.10', '0.10', '0.10'], ['0.10', '0.10', '0.11'], ['0.10', '0.10', '1.'], ['0.10', '0.11', '0.11'], ['0.10', '0.11', '1.'], ['0.10', '1.', '1.'], ['0.11', '0.11', '0.11'], ['0.11', '0.11', '1.'], ['0.11', '1.', '1.'], ['1.', '1.', '1.']]


In [ ]:
prob_sepf = [list(map(floatBin,i)) for i in prob_sep]
prob_range= [[i[0],i[1]-i[0],i[2]-i[1],1.0-i[2]] for i in prob_sepf]
print(prob_range)
sums = [sum(i) for i in prob_range]
print(sums)
print(len(prob_range))

[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.25, 0.75], [0.0, 0.0, 0.5, 0.5], [0.0, 0.0, 0.75, 0.25], [0.0, 0.0, 1.0, 0.0], [0.0, 0.25, 0.0, 0.75], [0.0, 0.25, 0.25, 0.5], [0.0, 0.25, 0.5, 0.25], [0.0, 0.25, 0.75, 0.0], [0.0, 0.5, 0.0, 0.5], [0.0, 0.5, 0.25, 0.25], [0.0, 0.5, 0.5, 0.0], [0.0, 0.75, 0.0, 0.25], [0.0, 0.75, 0.25, 0.0], [0.0, 1.0, 0, 0.0], [0.25, 0.0, 0.0, 0.75], [0.25, 0.0, 0.25, 0.5], [0.25, 0.0, 0.5, 0.25], [0.25, 0.0, 0.75, 0.0], [0.25, 0.25, 0.0, 0.5], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.5, 0.0], [0.25, 0.5, 0.0, 0.25], [0.25, 0.5, 0.25, 0.0], [0.25, 0.75, 0, 0.0], [0.5, 0.0, 0.0, 0.5], [0.5, 0.0, 0.25, 0.25], [0.5, 0.0, 0.5, 0.0], [0.5, 0.25, 0.0, 0.25], [0.5, 0.25, 0.25, 0.0], [0.5, 0.5, 0, 0.0], [0.75, 0.0, 0.0, 0.25], [0.75, 0.0, 0.25, 0.0], [0.75, 0.25, 0, 0.0], [1, 0, 0, 0.0]]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
35


В списке prob_range получили все возможные разбиения вероятностей перехода - строки матрицы переходных вероятностей - 35 вариантов в случае 2 знаков в двоичной записи переходных вероятностей. Далее перебираем для каждой из 4 строк матрицы переходных вероятностей все эти варианта и считаем вероятность получить заданную реализацию в рассматриваемой модели. Т.о. перебор идёт по $35^4 = 1 500 625$ вариантам

In [ ]:
digrams('01001111010')

(1, 3, 3, 3)

In [ ]:
# полный перебор
import time
from math import log
#s = '011110100010000111110'
s = '01001111010'
d = {'0':[0,1],'1':[2,3]}
mdl = 10**6 # minimum
mpr = -1 # max probability
start = time.time()
j = 0
for s1 in prob_range:
    for s2 in prob_range:
        for s3 in prob_range:
            for s4 in prob_range:
                P = [s1,s2,s3,s4]
                j += 1
                #print(P)
                dl = sum(len(binFloatPure(i)) for i in s1+s2+s3+s4) - 2*len(P)*len(P) # описательная длина - знаки посмле запятой в двоичной записи, убираем везде 0.
                pr = prob(P,d,s)
                dl -= log(pr,2) if pr > 0 else -10**6
                if pr > mpr:
                    mpr = pr
                    Pp = P
                    pdl = dl
                #print(pr)
                if dl == mdl:
                    P0.append(P)
                if dl < mdl:
                    mdl = dl
                    P0 = [P]
                    pr0 = pr
                if j % 5000 == 0:
                    end = time.time() - start
                    print(f'{j}, best prob: {pr0}, MDL: {mdl}, matrix: {P0}, time: {end}')
                #print(j,mdl,pr,dl,P)
print(f'По MDL: MDL = {mdl}, {P0}, p = {pr0}')
print(f'По вероятности: DL = {pdl}, {Pp}, p = {mpr}')
end = time.time() - start
print('Время выполнения:', end)
print(len(P0))
# для строки длины 20 за 6 мин 245 вариантов
# за 50 мин 2300 вариантов
# 1 ч 20 мин 3630 вариантов
# 2 ч + 1 ч 35 мин 9150 вариантов
# досчиталось до 11 734

5000, best prob: 0.0078125, MDL: 1000000, matrix: [[[0.0, 0.0, 0.0, 1.0], [0.0, 0.5, 0.5, 0.0], [0.5, 0.0, 0.5, 0.0], [0.0, 1.0, 0, 0.0]], [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0]], [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0]], [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 0, 0.0]], [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [1, 0, 0, 0.0]], [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]], [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0]], [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [0.0, 1.0, 0, 0.0]], [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [1, 0, 0, 0.0]], [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 0, 0.0], [0.0, 0.0, 0.0, 1.0]], [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 0, 0.0], [0.0

In [ ]:
matrs = [[[0.0, 0.0, 0.0, 1.0], [0.0, 0.5, 0.5, 0.0], [0.5, 0.0, 0.5, 0.0], [0.0, 1.0, 0, 0.0]], [[0.0, 0.0, 0.0, 1.0], [0.5, 0.0, 0.5, 0.0], [0.0, 1.0, 0, 0.0], [0.0, 0.0, 0.5, 0.5]], [[0.0, 0.0, 0.0, 1.0], [0.5, 0.0, 0.5, 0.0], [0.0, 1.0, 0, 0.0], [0.0, 0.5, 0.0, 0.5]], [[0.0, 0.0, 0.0, 1.0], [1, 0, 0, 0.0], [0.5, 0.0, 0.0, 0.5], [0.0, 0.5, 0.5, 0.0]], [[0.0, 0.0, 0.0, 1.0], [1, 0, 0, 0.0], [0.5, 0.0, 0.5, 0.0], [0.0, 0.5, 0.5, 0.0]], [[0.0, 0.0, 1.0, 0.0], [0.0, 0.5, 0.0, 0.5], [0.0, 1.0, 0, 0.0], [0.5, 0.0, 0.0, 0.5]], [[0.0, 0.0, 1.0, 0.0], [0.5, 0.0, 0.0, 0.5], [0.0, 0.0, 0.5, 0.5], [0.0, 1.0, 0, 0.0]], [[0.0, 0.0, 1.0, 0.0], [0.5, 0.0, 0.0, 0.5], [0.0, 0.5, 0.5, 0.0], [0.0, 1.0, 0, 0.0]], [[0.0, 0.0, 1.0, 0.0], [1, 0, 0, 0.0], [0.0, 0.5, 0.0, 0.5], [0.5, 0.0, 0.0, 0.5]], [[0.0, 0.0, 1.0, 0.0], [1, 0, 0, 0.0], [0.0, 0.5, 0.0, 0.5], [0.5, 0.0, 0.5, 0.0]], [[0.0, 0.5, 0.0, 0.5], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.5, 0.5], [1, 0, 0, 0.0]], [[0.0, 0.5, 0.0, 0.5], [0.0, 0.0, 1.0, 0.0], [0.5, 0.0, 0.5, 0.0], [1, 0, 0, 0.0]], [[0.0, 0.5, 0.5, 0.0], [0.0, 0.0, 0.0, 1.0], [1, 0, 0, 0.0], [0.0, 0.0, 0.5, 0.5]], [[0.0, 0.5, 0.5, 0.0], [0.0, 0.0, 0.0, 1.0], [1, 0, 0, 0.0], [0.5, 0.0, 0.0, 0.5]], [[0.0, 1.0, 0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.5, 0.0, 0.5], [0.5, 0.0, 0.5, 0.0]], [[0.0, 1.0, 0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.5, 0.5, 0.0], [0.5, 0.0, 0.5, 0.0]], [[0.0, 1.0, 0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.5, 0.0, 0.0, 0.5], [0.0, 0.5, 0.0, 0.5]], [[0.0, 1.0, 0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.5, 0.0, 0.0, 0.5], [0.0, 0.5, 0.5, 0.0]], [[0.5, 0.0, 0.0, 0.5], [0.0, 0.0, 1.0, 0.0], [1, 0, 0, 0.0], [0.0, 0.5, 0.0, 0.5]], [[0.5, 0.0, 0.5, 0.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.5, 0.5, 0.0], [1, 0, 0, 0.0]]]
for i in matrs:
    print(*i, sep = '\t')

[0.0, 0.0, 0.0, 1.0]	[0.0, 0.5, 0.5, 0.0]	[0.5, 0.0, 0.5, 0.0]	[0.0, 1.0, 0, 0.0]
[0.0, 0.0, 0.0, 1.0]	[0.5, 0.0, 0.5, 0.0]	[0.0, 1.0, 0, 0.0]	[0.0, 0.0, 0.5, 0.5]
[0.0, 0.0, 0.0, 1.0]	[0.5, 0.0, 0.5, 0.0]	[0.0, 1.0, 0, 0.0]	[0.0, 0.5, 0.0, 0.5]
[0.0, 0.0, 0.0, 1.0]	[1, 0, 0, 0.0]	[0.5, 0.0, 0.0, 0.5]	[0.0, 0.5, 0.5, 0.0]
[0.0, 0.0, 0.0, 1.0]	[1, 0, 0, 0.0]	[0.5, 0.0, 0.5, 0.0]	[0.0, 0.5, 0.5, 0.0]
[0.0, 0.0, 1.0, 0.0]	[0.0, 0.5, 0.0, 0.5]	[0.0, 1.0, 0, 0.0]	[0.5, 0.0, 0.0, 0.5]
[0.0, 0.0, 1.0, 0.0]	[0.5, 0.0, 0.0, 0.5]	[0.0, 0.0, 0.5, 0.5]	[0.0, 1.0, 0, 0.0]
[0.0, 0.0, 1.0, 0.0]	[0.5, 0.0, 0.0, 0.5]	[0.0, 0.5, 0.5, 0.0]	[0.0, 1.0, 0, 0.0]
[0.0, 0.0, 1.0, 0.0]	[1, 0, 0, 0.0]	[0.0, 0.5, 0.0, 0.5]	[0.5, 0.0, 0.0, 0.5]
[0.0, 0.0, 1.0, 0.0]	[1, 0, 0, 0.0]	[0.0, 0.5, 0.0, 0.5]	[0.5, 0.0, 0.5, 0.0]
[0.0, 0.5, 0.0, 0.5]	[0.0, 0.0, 1.0, 0.0]	[0.0, 0.0, 0.5, 0.5]	[1, 0, 0, 0.0]
[0.0, 0.5, 0.0, 0.5]	[0.0, 0.0, 1.0, 0.0]	[0.5, 0.0, 0.5, 0.0]	[1, 0, 0, 0.0]
[0.0, 0.5, 0.5, 0.0]	[0.0, 0.0, 0.0, 1.0